<a href="https://colab.research.google.com/github/PsyCharan17/pytorch-adventures/blob/main/01_pytorch_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt

Data preparing and loading

Linear regression implementation

In [ ]:
# create known parameters
weight = 0.7
bias = 0.3

# create
start =0
end =1
step =0.02
X = torch.arange(start,end ,step).unsqueeze(dim=1)
y = weight * X + bias


X[40:50] , y[40:50] , X.shape , y.shape

In [ ]:
train_split = int(0.8 *len(X))
X_train , y_train = X[:train_split] , y[:train_split]
X_test , y_test = X[train_split:] , y[train_split:]

In [ ]:
X_test , y_test

In [ ]:
def plot_predictions(train_data = X_train,
                     train_labels = y_train,
                     test_data = X_test,
                     test_labels = y_test,
                     predictions=None):
  plt.figure(figsize=(10,7))
  plt.scatter(train_data,train_labels,c="b",s=4,label="Training data")
  plt.scatter(test_data,test_labels,c="g",s=4,label="Testing data")
  if predictions is not None:
    plt.scatter(test_data,predictions,c="r",s=4, label="Predictions")

  plt.legend(prop={"size":14})


In [ ]:
plot_predictions()

In [ ]:
class LinearRegresionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.weights = nn.Parameter(torch.randn(1,requires_grad = True, dtype = torch.float))
    self.bias = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))

  def forward(self,x:torch.Tensor) -> torch.Tensor:
    return self.weights * x + self.bias



### Pytorch model building essentials
* torch.nn
* torch.nn.Parameter
* torch.nn.Module
* torch.optim
* def forward() - all nn.Module subclasses require you to overwrite forward() , this defines what happens in the forward computation

In [ ]:
torch.manual_seed(42)

model0 = LinearRegresionModel()
list(model0.parameters())

In [ ]:
model0.state_dict()

In [ ]:
# X_train,X_test

In [ ]:
#making predictions using torch.inference_mode()
with torch.inference_mode():
  y_preds = model0(X_test)
y_preds

In [ ]:
plot_predictions(predictions=y_preds)

In [ ]:
model0

In [ ]:
#setting up a loss function
loss_fn = nn.L1Loss()

#setting up stochacstic gradiient descent (SGD)
optimizer = torch.optim.SGD(params = model0.parameters(),lr=0.01)


### Building the training loop
* loop through the data
* forward pass (data moving through our model's forward() functions) to make predictions on data
* calculate the loss (compare the forward pass predictions to the ground truth)
* optimize zero grad
* loss backward - move through the network to calculate the gradients of each of the parameters of our model with respect to the loss ( backpropagation)
* optimizer step - use the optimizer to adjust model's parameters to try and reduce the loss (gradient descent)

In [ ]:
# building a training loop
%%time
epochs =200


epoch_count =[]
loss_values =[]
test_loss_values =[]

for epoch in range(epochs):
  model0.train()

  # 1. Forward pass
  y_pred = model0(X_train)

  # 2. Calculate the loss
  loss = loss_fn(y_pred,y_train)

  # 3. optimize zero grad
  optimizer.zero_grad()

  # 4. loss backward
  loss.backward()

  # 5. optimizer step
  optimizer.step()

  model0.eval() # turns off different settings not needed in the model not needed for evaluation/testing
  print(model0.state_dict())

  with torch.inference_mode():
    test_pred = model0(X_test)
    test_loss = loss_fn(test_pred,y_test)

    if epoch % 10 == 0:
      epoch_count.append(epoch)
      loss_values.append(loss)
      test_loss_values.append(test_loss)
      print(f"Epoch: {epoch} | Loss: {loss} | Test loss: {test_loss}")
      print(model0.state_dict())


In [ ]:
import numpy as np

# loss_values = np.array([lv.detach().cpu().numpy() for lv in loss_values]) #detach() detaches the tensor from the computation graph and cpu() moves the tensor to cpu if its on gpu
loss_values = torch.tensor(loss_values).cpu().numpy()
loss_values

In [ ]:
plt.plot(epoch_count,loss_values,label="Train Loss")
plt.plot(epoch_count,test_loss_values,label="Test Loss")
plt.title("Training and Testing Loss")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()

In [ ]:
weight , bias

In [ ]:
plot_predictions(predictions = test_pred)